### Predicting Kicker Points

In [3]:
import nfl_data_py as nfl
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
seasons = range(2020,2024)
pbp_df = nfl.import_pbp_data(seasons)

2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [5]:
pbp_df.columns.values

array(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       'side_of_field', 'yardline_100', 'game_date',
       'quarter_seconds_remaining', 'half_seconds_remaining',
       'game_seconds_remaining', 'game_half', 'quarter_end', 'drive',
       'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo',
       'ydsnet', 'desc', 'play_type', 'yards_gained', 'shotgun',
       'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble',
       'pass_length', 'pass_location', 'air_yards', 'yards_after_catch',
       'run_location', 'run_gap', 'field_goal_result', 'kick_distance',
       'extra_point_result', 'two_point_conv_result',
       'home_timeouts_remaining', 'away_timeouts_remaining', 'timeout',
       'timeout_team', 'td_team', 'td_player_name', 'td_player_id',
       'posteam_timeouts_remaining', 'defteam_timeouts_remaining',
       'total_home_score', 'total_away_score', 'posteam_score'

In [7]:
cols = ['game_id','home_team','away_team','yardline_100','game_seconds_remaining','ydstogo','play_type','kick_distance','extra_point_result','two_point_conv_result','extra_point_prob','two_point_conversion_prob','two_point_attempt','field_goal_attempt','kicker_player_name','defensive_two_point_attempt','defensive_extra_point_attempt', 'defensive_extra_point_conv', 'defensive_two_point_conv','roof', 'surface', 'temp', 'wind']
        

In [13]:
pbp_kicker = pbp_df[cols]
pbp_kicker = pbp_kicker.query("play_type == 'extra_point' | play_type == 'field_goal' ").reset_index()

In [18]:
pbp_kicker.tail(20)

,index,game_id,home_team,away_team,yardline_100,game_seconds_remaining,ydstogo,play_type,kick_distance,extra_point_result,...,field_goal_attempt,kicker_player_name,defensive_two_point_attempt,defensive_extra_point_attempt,defensive_extra_point_conv,defensive_two_point_conv,roof,surface,temp,wind
10675,223343,2023_10_SF_JAX,JAX,SF,15.0,760.0,0.0,extra_point,33.0,good,...,0.0,J.Moody,0.0,0.0,0.0,0.0,outdoors,grass,59.0,11.0
10676,223387,2023_10_TEN_TB,TB,TEN,20.0,3218.0,20.0,field_goal,38.0,None,...,1.0,N.Folk,0.0,0.0,0.0,0.0,outdoors,grass,85.0,6.0
10677,223410,2023_10_TEN_TB,TB,TEN,33.0,2564.0,2.0,field_goal,51.0,None,...,1.0,N.Folk,0.0,0.0,0.0,0.0,outdoors,grass,85.0,6.0
10678,223427,2023_10_TEN_TB,TB,TEN,15.0,2139.0,0.0,extra_point,33.0,good,...,0.0,C.McLaughlin,0.0,0.0,0.0,0.0,outdoors,grass,85.0,6.0
10679,223455,2023_10_TEN_TB,TB,TEN,7.0,1545.0,7.0,field_goal,25.0,None,...,1.0,C.McLaughlin,0.0,0.0,0.0,0.0,outdoors,grass,85.0,6.0
10680,223473,2023_10_TEN_TB,TB,TEN,15.0,1066.0,0.0,extra_point,33.0,good,...,0.0,C.McLaughlin,0.0,0.0,0.0,0.0,outdoors,grass,85.0,6.0
10681,223488,2023_10_TEN_TB,TB,TEN,9.0,777.0,9.0,field_goal,27.0,None,...,1.0,N.Folk,0.0,0.0,0.0,0.0,outdoors,grass,85.0,6.0
10682,223503,2023_10_TEN_TB,TB,TEN,17.0,425.0,9.0,field_goal,35.0,None,...,1.0,C.McLaughlin,0.0,0.0,0.0,0.0,outdoors,grass,85.0,6.0
10683,223541,2023_10_WAS_SEA,SEA,WAS,15.0,3506.0,0.0,extra_point,33.0,failed,...,0.0,J.Slye,0.0,0.0,0.0,0.0,outdoors,fieldturf,52.0,4.0
10684,223554,2023_10_WAS_SEA,SEA,WAS,26.0,3228.0,6.0,field_goal,45.0,None,...,1.0,J.Myers,0.0,0.0,0.0,0.0,outdoors,fieldturf,52.0,4.0


In [ ]:
### Exploratory Data Analysis
#### Exploring how distance, roof, surface, temp, and wind affect kicker outcomes

In [ ]:
#### If roof = indoors does that cancel out other variables?
